# Búsqueda de autores en Orcid
See [search-orcid-registry](http://members.orcid.org/api/tutorial/search-orcid-registry)

ORCID: https://github.com/ORCID/python-orcid

In [30]:
%%writefile drive.cfg
[FILES]
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs 

Writing drive.cfg


In [32]:
import requests
import pandas as pd
import wosplus as wp

In [33]:
bdi=wp.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [34]:
bdi[:2]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación
1,71652139,Carlos Mario Jaramillo Ramirez,Carlos Mario,Jaramillo,Ramirez,Facultad de Artes,Departamento de Artes Visuales,DIDARTE. Didáctica de las artes


In [78]:
# From https://github.com/ORCID/python-orcid
import orcid
api = orcid.PublicAPI('APP-R8J5P57TTE983P54', 'c6e1dd48-80d3-4dd9-9336-fc41728ac8fe', sandbox=False)
#token = api.get_token_from_authorization_code(authorization_code,
#                                              redirect_uri)


In [79]:
search_token = api.get_search_token_from_orcid()

In [80]:
search_token

'7e328e49-ee8c-4788-b070-27df3a9a3dea'

In [ ]:
https://pub.orcid.org

In [81]:
HEADERS_orcid={'Content-type': 'application/vnd.orcid+json',
    'Access token': search_token}

In [82]:
r=requests.get('https://pub.orcid.org/v2.1/search/?q=newman', headers=HEADERS_orcid)

In [86]:
pd.read_json(r.text)

,num-found,result
0,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0002-7182-6999', 'uri': 'https://orcid.org/0000-0002-7182-6999'}}"
1,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0002-5662-263X', 'uri': 'https://orcid.org/0000-0002-5662-263X'}}"
2,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0002-0675-9611', 'uri': 'https://orcid.org/0000-0002-0675-9611'}}"
3,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0003-1948-0385', 'uri': 'https://orcid.org/0000-0003-1948-0385'}}"
4,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0003-4664-308X', 'uri': 'https://orcid.org/0000-0003-4664-308X'}}"
5,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0003-4701-7523', 'uri': 'https://orcid.org/0000-0003-4701-7523'}}"
6,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0002-3422-3523', 'uri': 'https://orcid.org/0000-0002-3422-3523'}}"
7,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0002-5876-4498', 'uri': 'https://orcid.org/0000-0002-5876-4498'}}"
8,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0001-8875-0655', 'uri': 'https://orcid.org/0000-0001-8875-0655'}}"
9,871,"{'orcid-identifier': {'host': 'orcid.org', 'path': '0000-0003-0534-8914', 'uri': 'https://orcid.org/0000-0003-0534-8914'}}"


In [37]:
import pandas as pd
def search_orcid(q='family-name:Taylor+AND+given-names:Michael'):
    '''
    Search API orcid in json for query q: (see examples here: http://members.orcid.org/api/tutorial/search-orcid-registry)
    Example: 
    q=family-name:Taylor+AND+given-names:Michael
    '''
    r=requests.get('https://pub.sandbox.orcid.org/v2.1/search/?q={}'.format(q),
                   headers={'Content-type': 'application/vnd.orcid+json'})
    if r.status_code==200:
        return pd.read_json(r.text)
    else:
        return pd.DataFrame()

## Formulas de búsqueda

In [50]:
search_orcid('family-name:Restrepo')#+AND+given-names:Diego')#+AND+affiliation-org-name:Universidad+de+Antioquia')

,num-found,result
0,1,"{'orcid-identifier': {'host': 'sandbox.orcid.org', 'path': '0000-0002-7109-6893', 'uri': 'https://sandbox.orcid.org/0000-0002-7109-6893'}}"


## Appendix

In [17]:
r=requests.get('http://docs.python-requests.org/en/master/')

In [23]:
print( r.text[:193] )


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
  <head>
    
